In [1]:
import os
import numpy as np
import pydicom
import imageio
print('libraries loaded')

libraries loaded


In [2]:
def convert_to_uint16(data, source_max):
    target_max = 65535  # 2 ** 16 - 1
    # build a linear lookup table (LUT) indexed from 0 to source_max
    source_range = np.arange(source_max + 1)
    lut = np.round(source_range * target_max / source_max).astype(np.uint16)
    # apply it
    return lut[data]

def should_invert_contrast(pixel_data, threshold=32767):
    # Extract top-left corner pixel
    top_left_pixel = pixel_data[1, 200]

    # Calculate the intensity of the center pixel
    center_pixel_intensity = pixel_data[pixel_data.shape[0] // 2, pixel_data.shape[1] // 2]

    # Check if the center pixel is darker than the top-left corner pixel
    return center_pixel_intensity < top_left_pixel

def convert_dicom_to_tiff(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through DICOM files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.dicom'):
            dicom_path = os.path.join(input_folder, filename)

            try:
                dicom = pydicom.dcmread(dicom_path, force=True)
            except Exception as e:
                print(f" Read Error DICOM file {dicom_path}: {e}")
                continue

            # Check if Pixel Data is present
            if 'PixelData' not in dicom:
                print(f"Error: Pixel Data not found in DICOM file {dicom_path}")
                continue

            # Set TransferSyntaxUID if not present
            if not hasattr(dicom.file_meta, 'TransferSyntaxUID'):
                dicom.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

            try:
                image = dicom.pixel_array
                pixel_data = image.astype(np.int32)  # Ensure pixel data is integer

                # Check if inversion is needed based on the center pixel
                if should_invert_contrast(pixel_data, threshold=32767):
                    pixel_data = np.max(pixel_data) - pixel_data  # Invert contrast

                pixel_data = convert_to_uint16(pixel_data, np.max(pixel_data))  # Convert to uint16

                # Save as TIFF in the output folder
                output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + '.tiff')
                imageio.imwrite(output_path, pixel_data)

            except Exception as e:
                print(f"Error processing DICOM file {dicom_path}: {e}")
                continue

In [3]:
if __name__ == "__main__":
    # Convert test images
    # test_input_folder = "C:/Users/Alex/Desktop/SENECA/Semester 9/BDC800_Capstone/Capstone Project/DeepRadiology/dataset/images/test_dicom"
    # test_output_folder = "C:/Users/Alex/Desktop/SENECA/Semester 9/BDC800_Capstone/Capstone Project/DeepRadiology/dataset/images/test"
    # convert_dicom_to_tiff(test_input_folder, test_output_folder)

    # Convert train images
    input_folder = "C:/Users/Alex/Desktop/SENECA/Semester 9/BDC800_Capstone/Capstone Project/DeepRadiology/dataset/images/train_dicom_og"
    output_folder = "C:/Users/Alex/Desktop/SENECA/Semester 9/BDC800_Capstone/Capstone Project/DeepRadiology/dataset/images/train_og2"
    convert_dicom_to_tiff(input_folder, output_folder)

    print('Complete')

Error: Pixel Data not found in DICOM file C:/Users/Alex/Desktop/SENECA/Semester 9/BDC800_Capstone/Capstone Project/DeepRadiology/dataset/images/train_dicom_og\000434271f63a053c4128a0ba6352c7f.dicom


C:\Users\Alex\anaconda3\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
C:\Users\Alex\anaconda3\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:238: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


Complete
